In [6]:
import tensorflow as tf
import numpy as np
import cv2
from keras.utils import Sequence
from keras.applications.mobilenet_v3 import preprocess_input
from glob import glob
import os
from sklearn.model_selection import train_test_split

from keras.applications import MobileNetV3Large
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import TopKCategoricalAccuracy
from keras.layers import RandomRotation
from keras.callbacks import LearningRateScheduler



import math
import json


import re
print(tf.__version__)
import tensorflow_probability as tfp
tfd = tfp.distributions


2.15.0




In [7]:
# Load class names
with open("food-101/meta/classes.txt", "r") as file:
    classes = [line.strip() for line in file]

# Read image paths
with open("food-101/meta/train.txt", "r") as file:
    train_paths = [line.strip() for line in file]

with open("food-101/meta/test.txt", "r") as file:
    test_paths = [line.strip() for line in file]

# Helper function to convert label format
def to_snake_case(name):
    return name.replace(" ", "_").lower()

# Convert paths to full paths
train_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in train_paths]
train_labels = [to_snake_case(p.split("/")[0]) for p in train_paths]

test_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in test_paths]
test_labels = [to_snake_case(p.split("/")[0]) for p in test_paths]

# Map labels to indices
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
train_label_indices = [class_to_index[label] for label in train_labels]
test_label_indices = [class_to_index[label] for label in test_labels]

# ✂️ Split 80% train, 20% val
train_paths_split, val_paths_split, train_labels_split, val_labels_split = train_test_split(
    train_full_paths, train_label_indices, test_size=0.2, stratify=train_label_indices, random_state=42
)

# Constants
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = len(classes)

rotation_layer = RandomRotation(factor=0.014)

# Augmentation function
def augment_image(img):
    img = rotation_layer(img)
    img = tf.image.resize_with_crop_or_pad(img, 256, 256)
    img = tf.image.random_crop(img, size=[224, 224, 3])    
    img = tf.image.random_flip_left_right(img)            
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.image.random_saturation(img, 0.9, 1.1)
    img = tf.image.random_hue(img, 0.08)
    img = tf.clip_by_value(img, 0.0, 255.0)
    return img

# Preprocess each sample
def process_example(path, label, augment=False):
    img_raw = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    if augment:
        img = augment_image(img)
    img = preprocess_input(img)
    label = tf.one_hot(label, NUM_CLASSES)
    return img, label

# Dataset creation function
def create_dataset(paths, labels, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.map(lambda x, y: process_example(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

val_ds = create_dataset(val_paths_split, val_labels_split, augment=False, shuffle=False)
test_ds = create_dataset(test_full_paths, test_label_indices, shuffle=False)

In [9]:
def get_cosine_annealing_with_restarts(
    initial_lr=1e-3,
    min_lr=1e-7,
    warmup_epochs=5,
    T_0=20,
    T_mult=1
):
    def scheduler(epoch, lr):

        # Warmup phase
        if epoch < warmup_epochs:
            return float(initial_lr * (epoch + 1) / warmup_epochs)

        # Cosine annealing with restarts
        t = epoch - warmup_epochs  # Epochs after warmup
        T_i = T_0
        cumulative = 0

        while t >= T_i:
            t -= T_i
            cumulative += T_i
            T_i *= T_mult

        progress = t / T_i
        cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))
        return float(min_lr + (initial_lr - min_lr) * cosine_decay)

    return scheduler

In [11]:
def apply_cutmix(ds, alpha=1.0):
    def cutmix(images, labels):
        batch_size = tf.shape(images)[0]
        indices = tf.random.shuffle(tf.range(batch_size))
        image1, image2 = images, tf.gather(images, indices)
        label1, label2 = labels, tf.gather(labels, indices)

        h = tf.shape(images)[1]
        w = tf.shape(images)[2]
        lam = tfd.Beta(alpha, alpha).sample()

        cut_rat = tf.math.sqrt(1. - lam)
        cut_w = tf.cast(tf.cast(w, tf.float32) * cut_rat, tf.int32)
        cut_h = tf.cast(tf.cast(h, tf.float32) * cut_rat, tf.int32)

        cx = tf.random.uniform([], 0, w, dtype=tf.int32)
        cy = tf.random.uniform([], 0, h, dtype=tf.int32)

        x1 = tf.clip_by_value(cx - cut_w // 2, 0, w)
        y1 = tf.clip_by_value(cy - cut_h // 2, 0, h)
        x2 = tf.clip_by_value(cx + cut_w // 2, 0, w)
        y2 = tf.clip_by_value(cy + cut_h // 2, 0, h)

        def replace_patch(img1, img2):
            patch = img2[y1:y2, x1:x2, :]
            paddings = [[y1, h - y2], [x1, w - x2], [0, 0]]
            patch_padded = tf.pad(patch, paddings)
            mask = tf.pad(tf.ones_like(patch), paddings)
            return tf.where(mask > 0, patch_padded, img1)

        mixed_images = tf.map_fn(lambda x: replace_patch(x[0], x[1]), (image1, image2), dtype=tf.float32)

        patch_area = tf.cast((x2 - x1) * (y2 - y1), tf.float32)
        total_area = tf.cast(h * w, tf.float32)
        lam_adjusted = 1.0 - (patch_area / total_area)

        mixed_labels = lam_adjusted * label1 + (1.0 - lam_adjusted) * label2

        return mixed_images, mixed_labels

    return ds.map(cutmix, num_parallel_calls=tf.data.AUTOTUNE)


def apply_mixup(ds, alpha):
    def mixup(images, labels):
        batch_size = tf.shape(images)[0]
        indices = tf.random.shuffle(tf.range(batch_size))
        image1, image2 = images, tf.gather(images, indices)
        label1, label2 = labels, tf.gather(labels, indices)

        lam = tfd.Beta(alpha, alpha).sample()
        lam_x = tf.reshape(lam, [1, 1, 1])
        lam_y = tf.reshape(lam, [1])

        mixed_image = lam_x * image1 + (1 - lam_x) * image2
        mixed_label = lam_y * label1 + (1 - lam_y) * label2

        return mixed_image, mixed_label

    return ds.map(mixup, num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
# No Augmentation
train_ds_prep = create_dataset(train_paths_split, train_labels_split, augment=False)
train_ds = apply_mixup(train_ds_prep, 1)

loss_fn = CategoricalCrossentropy(label_smoothing=0.05)


base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu')(x)
output = Dense(101, activation='softmax')(x)

checkpoint_cb = ModelCheckpoint(
    filepath='D:\\Skripsi\\MobilNetV3-Food101-Rev-Baseline\\Model_MobileNetV3-2-MixUp1.keras',          
    monitor='val_accuracy',            
    mode='max',                         
    save_best_only=True,                            
    verbose=1
)

lr_scheduler = LearningRateScheduler(get_cosine_annealing_with_restarts())

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5, 
    callbacks=[checkpoint_cb, lr_scheduler]
)

total_layers = len(base_model.layers)

unfreeze_count = int(total_layers * 0.3)

for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history_fine_tune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    initial_epoch=5,
    callbacks=[checkpoint_cb, lr_scheduler]
)

combined_history = {}
for key in history.history.keys():
    combined_history[key] = history.history[key] + history_fine_tune.history[key]

with open('skripsi/history/Model_History_MobilNetV3-2-MixUp1.json', 'w') as f:
    json.dump(combined_history, f)


Epoch 1/5

947/947 [==============================] - ETA: 0s - loss: 3.9751 - accuracy: 0.1850 - top_5_accuracy: 0.3878
Epoch 1: val_accuracy improved from -inf to 0.44878, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 759s 798ms/step - loss: 3.9751 - accuracy: 0.1850 - top_5_accuracy: 0.3878 - val_loss: 2.5954 - val_accuracy: 0.4488 - val_top_5_accuracy: 0.7241 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 3.3769 - accuracy: 0.3499 - top_5_accuracy: 0.6148
Epoch 2: val_accuracy improved from 0.44878 to 0.54290, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 739s 780ms/step - loss: 3.3769 - accuracy: 0.3499 - top_5_accuracy: 0.6148 - val_loss: 2.1590 - val_accuracy: 0.5429 - val_top_5_accuracy: 0.8036 - lr: 4.0000e-04
Epoch 3/5
947/947 [==============================] - 

TypeError: Object of type float32 is not JSON serializable

WARNING:tensorflow:From c:\Users\Lucas Salvacio\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\normalization\batch_normalization.py:979: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

Epoch 1/5
WARNING:tensorflow:From c:\Users\Lucas Salvacio\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

947/947 [==============================] - ETA: 0s - loss: 3.9751 - accuracy: 0.1850 - top_5_accuracy: 0.3878
Epoch 1: val_accuracy improved from -inf to 0.44878, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 759s 798ms/step - loss: 3.9751 - accuracy: 0.1850 - top_5_accuracy: 0.3878 - val_loss: 2.5954 - val_accuracy: 0.4488 - val_top_5_accuracy: 0.7241 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 3.3769 - accuracy: 0.3499 - top_5_accuracy: 0.6148
Epoch 2: val_accuracy improved from 0.44878 to 0.54290, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 739s 780ms/step - loss: 3.3769 - accuracy: 0.3499 - top_5_accuracy: 0.6148 - val_loss: 2.1590 - val_accuracy: 0.5429 - val_top_5_accuracy: 0.8036 - lr: 4.0000e-04
Epoch 3/5
947/947 [==============================] - ETA: 0s - loss: 3.2594 - accuracy: 0.3925 - top_5_accuracy: 0.6560
Epoch 3: val_accuracy improved from 0.54290 to 0.57195, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 755s 796ms/step - loss: 3.2594 - accuracy: 0.3925 - top_5_accuracy: 0.6560 - val_loss: 2.0457 - val_accuracy: 0.5719 - val_top_5_accuracy: 0.8213 - lr: 6.0000e-04
Epoch 4/5
947/947 [==============================] - ETA: 0s - loss: 3.2203 - accuracy: 0.4088 - top_5_accuracy: 0.6693
Epoch 4: val_accuracy improved from 0.57195 to 0.57446, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 787s 830ms/step - loss: 3.2203 - accuracy: 0.4088 - top_5_accuracy: 0.6693 - val_loss: 2.0448 - val_accuracy: 0.5745 - val_top_5_accuracy: 0.8247 - lr: 8.0000e-04
Epoch 5/5
947/947 [==============================] - ETA: 0s - loss: 3.1566 - accuracy: 0.4256 - top_5_accuracy: 0.6850
Epoch 5: val_accuracy improved from 0.57446 to 0.58370, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 787s 830ms/step - loss: 3.1566 - accuracy: 0.4256 - top_5_accuracy: 0.6850 - val_loss: 1.9900 - val_accuracy: 0.5837 - val_top_5_accuracy: 0.8302 - lr: 0.0010
Epoch 6/25
947/947 [==============================] - ETA: 0s - loss: 3.0852 - accuracy: 0.4601 - top_5_accuracy: 0.7093
Epoch 6: val_accuracy did not improve from 0.58370
947/947 [==============================] - 1033s 1s/step - loss: 3.0852 - accuracy: 0.4601 - top_5_accuracy: 0.7093 - val_loss: 4.1989 - val_accuracy: 0.3131 - val_top_5_accuracy: 0.5855 - lr: 0.0010
Epoch 7/25
947/947 [==============================] - ETA: 0s - loss: 2.8373 - accuracy: 0.5404 - top_5_accuracy: 0.7753
Epoch 7: val_accuracy did not improve from 0.58370
947/947 [==============================] - 1012s 1s/step - loss: 2.8373 - accuracy: 0.5404 - top_5_accuracy: 0.7753 - val_loss: 2.1930 - val_accuracy: 0.5538 - val_top_5_accuracy: 0.8026 - lr: 9.9384e-04
Epoch 8/25
947/947 [==============================] - ETA: 0s - loss: 2.7828 - accuracy: 0.5733 - top_5_accuracy: 0.7950
Epoch 8: val_accuracy improved from 0.58370 to 0.63624, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1020s 1s/step - loss: 2.7828 - accuracy: 0.5733 - top_5_accuracy: 0.7950 - val_loss: 1.8969 - val_accuracy: 0.6362 - val_top_5_accuracy: 0.8625 - lr: 9.7553e-04
Epoch 9/25
947/947 [==============================] - ETA: 0s - loss: 2.6446 - accuracy: 0.6085 - top_5_accuracy: 0.8161
Epoch 9: val_accuracy improved from 0.63624 to 0.67142, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1033s 1s/step - loss: 2.6446 - accuracy: 0.6085 - top_5_accuracy: 0.8161 - val_loss: 1.7418 - val_accuracy: 0.6714 - val_top_5_accuracy: 0.8800 - lr: 9.4551e-04
Epoch 10/25
947/947 [==============================] - ETA: 0s - loss: 2.6048 - accuracy: 0.6281 - top_5_accuracy: 0.8258
Epoch 10: val_accuracy improved from 0.67142 to 0.68594, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1034s 1s/step - loss: 2.6048 - accuracy: 0.6281 - top_5_accuracy: 0.8258 - val_loss: 1.6785 - val_accuracy: 0.6859 - val_top_5_accuracy: 0.8870 - lr: 9.0452e-04
Epoch 11/25
947/947 [==============================] - ETA: 0s - loss: 2.4809 - accuracy: 0.6596 - top_5_accuracy: 0.8479
Epoch 11: val_accuracy improved from 0.68594 to 0.70455, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1018s 1s/step - loss: 2.4809 - accuracy: 0.6596 - top_5_accuracy: 0.8479 - val_loss: 1.5953 - val_accuracy: 0.7046 - val_top_5_accuracy: 0.8959 - lr: 8.5357e-04
Epoch 12/25
947/947 [==============================] - ETA: 0s - loss: 2.4922 - accuracy: 0.6662 - top_5_accuracy: 0.8496
Epoch 12: val_accuracy improved from 0.70455 to 0.71512, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1017s 1s/step - loss: 2.4922 - accuracy: 0.6662 - top_5_accuracy: 0.8496 - val_loss: 1.5643 - val_accuracy: 0.7151 - val_top_5_accuracy: 0.8999 - lr: 7.9391e-04
Epoch 13/25
947/947 [==============================] - ETA: 0s - loss: 2.4577 - accuracy: 0.6807 - top_5_accuracy: 0.8556
Epoch 13: val_accuracy improved from 0.71512 to 0.72330, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1031s 1s/step - loss: 2.4577 - accuracy: 0.6807 - top_5_accuracy: 0.8556 - val_loss: 1.5275 - val_accuracy: 0.7233 - val_top_5_accuracy: 0.9036 - lr: 7.2702e-04
Epoch 14/25
947/947 [==============================] - ETA: 0s - loss: 2.3890 - accuracy: 0.7032 - top_5_accuracy: 0.8682
Epoch 14: val_accuracy improved from 0.72330 to 0.73498, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1014s 1s/step - loss: 2.3890 - accuracy: 0.7032 - top_5_accuracy: 0.8682 - val_loss: 1.4799 - val_accuracy: 0.7350 - val_top_5_accuracy: 0.9077 - lr: 6.5454e-04
Epoch 15/25
947/947 [==============================] - ETA: 0s - loss: 2.3576 - accuracy: 0.7165 - top_5_accuracy: 0.8742
Epoch 15: val_accuracy improved from 0.73498 to 0.74548, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1010s 1s/step - loss: 2.3576 - accuracy: 0.7165 - top_5_accuracy: 0.8742 - val_loss: 1.4290 - val_accuracy: 0.7455 - val_top_5_accuracy: 0.9141 - lr: 5.7826e-04
Epoch 16/25
947/947 [==============================] - ETA: 0s - loss: 2.2916 - accuracy: 0.7336 - top_5_accuracy: 0.8819
Epoch 16: val_accuracy did not improve from 0.74548
947/947 [==============================] - 1023s 1s/step - loss: 2.2916 - accuracy: 0.7336 - top_5_accuracy: 0.8819 - val_loss: 1.4366 - val_accuracy: 0.7453 - val_top_5_accuracy: 0.9148 - lr: 5.0005e-04
Epoch 17/25
947/947 [==============================] - ETA: 0s - loss: 2.3241 - accuracy: 0.7348 - top_5_accuracy: 0.8810
Epoch 17: val_accuracy improved from 0.74548 to 0.74851, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1010s 1s/step - loss: 2.3241 - accuracy: 0.7348 - top_5_accuracy: 0.8810 - val_loss: 1.4231 - val_accuracy: 0.7485 - val_top_5_accuracy: 0.9172 - lr: 4.2184e-04
Epoch 18/25
947/947 [==============================] - ETA: 0s - loss: 2.2605 - accuracy: 0.7533 - top_5_accuracy: 0.8915
Epoch 18: val_accuracy improved from 0.74851 to 0.75558, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1019s 1s/step - loss: 2.2605 - accuracy: 0.7533 - top_5_accuracy: 0.8915 - val_loss: 1.3868 - val_accuracy: 0.7556 - val_top_5_accuracy: 0.9190 - lr: 3.4556e-04
Epoch 19/25
947/947 [==============================] - ETA: 0s - loss: 2.2676 - accuracy: 0.7514 - top_5_accuracy: 0.8873
Epoch 19: val_accuracy improved from 0.75558 to 0.75716, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1019s 1s/step - loss: 2.2676 - accuracy: 0.7514 - top_5_accuracy: 0.8873 - val_loss: 1.3906 - val_accuracy: 0.7572 - val_top_5_accuracy: 0.9187 - lr: 2.7308e-04
Epoch 20/25
947/947 [==============================] - ETA: 0s - loss: 2.1372 - accuracy: 0.7820 - top_5_accuracy: 0.9045
Epoch 20: val_accuracy improved from 0.75716 to 0.76422, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1025s 1s/step - loss: 2.1372 - accuracy: 0.7820 - top_5_accuracy: 0.9045 - val_loss: 1.3606 - val_accuracy: 0.7642 - val_top_5_accuracy: 0.9199 - lr: 2.0619e-04
Epoch 21/25
947/947 [==============================] - ETA: 0s - loss: 2.1207 - accuracy: 0.7924 - top_5_accuracy: 0.9097
Epoch 21: val_accuracy did not improve from 0.76422
947/947 [==============================] - 1019s 1s/step - loss: 2.1207 - accuracy: 0.7924 - top_5_accuracy: 0.9097 - val_loss: 1.3520 - val_accuracy: 0.7642 - val_top_5_accuracy: 0.9213 - lr: 1.4653e-04
Epoch 22/25
947/947 [==============================] - ETA: 0s - loss: 2.1282 - accuracy: 0.7918 - top_5_accuracy: 0.9091
Epoch 22: val_accuracy improved from 0.76422 to 0.76587, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1026s 1s/step - loss: 2.1282 - accuracy: 0.7918 - top_5_accuracy: 0.9091 - val_loss: 1.3510 - val_accuracy: 0.7659 - val_top_5_accuracy: 0.9212 - lr: 9.5582e-05
Epoch 23/25
947/947 [==============================] - ETA: 0s - loss: 2.1572 - accuracy: 0.7860 - top_5_accuracy: 0.9025
Epoch 23: val_accuracy did not improve from 0.76587
947/947 [==============================] - 1022s 1s/step - loss: 2.1572 - accuracy: 0.7860 - top_5_accuracy: 0.9025 - val_loss: 1.3622 - val_accuracy: 0.7650 - val_top_5_accuracy: 0.9221 - lr: 5.4591e-05
Epoch 24/25
947/947 [==============================] - ETA: 0s - loss: 2.1921 - accuracy: 0.7819 - top_5_accuracy: 0.9010
Epoch 24: val_accuracy did not improve from 0.76587
947/947 [==============================] - 1020s 1s/step - loss: 2.1921 - accuracy: 0.7819 - top_5_accuracy: 0.9010 - val_loss: 1.3605 - val_accuracy: 0.7657 - val_top_5_accuracy: 0.9222 - lr: 2.4569e-05
Epoch 25/25
947/947 [==============================] - ETA: 0s - loss: 2.1020 - accuracy: 0.8062 - top_5_accuracy: 0.9152
Epoch 25: val_accuracy improved from 0.76587 to 0.76627, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-2-MixUp1.keras
947/947 [==============================] - 1022s 1s/step - loss: 2.1020 - accuracy: 0.8062 - top_5_accuracy: 0.9152 - val_loss: 1.3548 - val_accuracy: 0.7663 - val_top_5_accuracy: 0.9225 - lr: 6.2552e-06

In [13]:
results = model.evaluate(test_ds)

395/395 [==============================] - 310s 783ms/step - loss: 1.1597 - accuracy: 0.8197 - top_5_accuracy: 0.9533
